<a href="https://colab.research.google.com/github/MichaelAnth/ML-for-Digital-Advertising-Applications/blob/master/Python_Exercise_4_Census_Reporter_for_Geographic_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digital Advertising Python Exercise 4
### **Author:** Michael Anthony
### **Date:** December 2019
### **Objective:** Use the Census Reporter API to make geographic selections for areas with demographic features of interest for the marketing of Ozo coffee products.


# Document Setup and Loading Libraries

In [0]:
import pandas as pd
import pandas
# sklearn.cross_validation was not importing train_test_split so the model
# selection module is used instead
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoLarsCV
import matplotlib.pyplot as plt

# Import and View Data

In [175]:
# Load Data
DATA_URL = 'https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv'
df = pd.read_csv(DATA_URL)
# view data frame
df.head()

,CBSA,NAME,POPESTIMATE2017
0,10180,"Abilene, TX",170219
1,10420,"Akron, OH",703505
2,10500,"Albany, GA",151434
3,10540,"Albany, OR",125047
4,10580,"Albany-Schenectady-Troy, NY",886188


Lets view the 10 US metro areas with the greatest population.

In [176]:
df.sort_values(by=['POPESTIMATE2017'],ascending=False).head(4)

,CBSA,NAME,POPESTIMATE2017
249,35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876
210,31080,"Los Angeles-Long Beach-Anaheim, CA",13353907
68,16980,"Chicago-Naperville-Elgin, IL-IN-WI",9533040
86,19100,"Dallas-Fort Worth-Arlington, TX",7399662


#Demographics of interest
Based on a published market analysis of premium coffee drinker preferences we can draw a few key insights. The first is that high socio-economic status (SES) consumers particularly care about the taste of their coffee and are looking for superior taste when choosing to purchase premium brands. Although we can assume our Ozo coffee has a superior taste, the dominant diferentiator for Ozo is that it is organic. This quality is most appealing for the second highest income bracket in our analysis, **50-75k**. Hence, we should target cities where the proportion of the population in this income bracket is higher than in other cities.

Our product is organic and contains no artificial ingredients which are appealing qualities for **18-34** year old. Hence we should look for areas where the proportion of the population in the 18 to 34 age range is particularly high.

Next, we notice that these apeals to healthy and environmental friendly products are particularly relevant for hispanic populations, hence we may target cities where there is a particularly **large hispanic population**.

# Deterine the Target Tables
We must determine which data sets to pull from census reporter.
1. For the proportion of the population in each income bracket: Table **B19001**.
2. For a breakdown of population by age group: Table **B01001**
3. For the proportion of the population that is hispanic: Table **B03002** 


In [0]:
tableids = ['B19001', 'B01001', 'B03002']

In [178]:
geoid = '31000US' + df['CBSA'].astype(str)
df['geoid'] = geoid
geoid=list(geoid)
print(geoid[0:5])

['31000US10180', '31000US10420', '31000US10500', '31000US10540', '31000US10580']


In [0]:
## get the data for each parameter of interest as a separate url
# url_list=[]
# for i in tableids:
#     for j in geoid:
#         tablestring=str(i)
#         geographytag=str(j)
#         requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=310|01000US' % (tablestring, geographytag)
#         url_list.append(requesturl)
# print(url_list)


In [179]:
# Or we can get the data for all our paramaters with one url
concat_tableid_string = ','.join(tableids)
requested_url = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=310|01000US' % concat_tableid_string
print(requested_url)

https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03002&geo_ids=310|01000US


In [0]:
import requests 
import json
loadedjson = requests.get(requested_url)
parsedjson = loadedjson.json()

In [181]:
#view the keys of the json dictionary to see how the data is organized
parsedjson.keys()

dict_keys(['release', 'tables', 'data', 'geography'])

In [0]:
## View the structure of the dictionary to find the nessesary values
#parsedjson

In [183]:
# print(parsedjson['31000US10180'])
print(parsedjson.get('tables')['B03002'])
# print(parsedjson.get('data')['estimate'])
parsedjson['data']['31000US10180']['B01001']['estimate']['B01001001']

{'title': 'Hispanic or Latino Origin by Race', 'universe': 'Total Population', 'denominator_column_id': 'B03002001', 'columns': {'B03002001': {'name': 'Total:', 'indent': 0}, 'B03002002': {'name': 'Not Hispanic or Latino:', 'indent': 1}, 'B03002003': {'name': 'White alone', 'indent': 2}, 'B03002004': {'name': 'Black or African American alone', 'indent': 2}, 'B03002005': {'name': 'American Indian and Alaska Native alone', 'indent': 2}, 'B03002006': {'name': 'Asian alone', 'indent': 2}, 'B03002007': {'name': 'Native Hawaiian and Other Pacific Islander alone', 'indent': 2}, 'B03002008': {'name': 'Some other race alone', 'indent': 2}, 'B03002009': {'name': 'Two or more races:', 'indent': 2}, 'B03002010': {'name': 'Two races including Some other race', 'indent': 3}, 'B03002011': {'name': 'Two races excluding Some other race, and three or more races', 'indent': 3}, 'B03002012': {'name': 'Hispanic or Latino:', 'indent': 1}, 'B03002013': {'name': 'White alone', 'indent': 2}, 'B03002014': {'nam

169000.0

#Variables of interest

1. income: **50-75k**. 

2. age: **18-34**

3. ethnicity: **large hispanic population**.


To find the city with the largest total target population, we want to take the sum of the sub populations of interest and find the city with the largest in market audience. We begin by identify the tables containing the subpopulation data we are interested in aggregating.

In [0]:
# population size by Age tables:
    #male
    # 'B01001007': {'indent': 2, 'name': '18 and 19 years'},
    # 'B01001008': {'indent': 2, 'name': '20 years'},
    # 'B01001009': {'indent': 2, 'name': '21 years'},
    # 'B01001010': {'indent': 2, 'name': '22 to 24 years'},
    # 'B01001011': {'indent': 2, 'name': '25 to 29 years'},
    # 'B01001012': {'indent': 2, 'name': '30 to 34 years'},
    #female
    # 'B01001031': {'indent': 2, 'name': '18 and 19 years'},
    # 'B01001032': {'indent': 2, 'name': '20 years'},
    # 'B01001033': {'indent': 2, 'name': '21 years'},
    # 'B01001034': {'indent': 2, 'name': '22 to 24 years'},
    # 'B01001035': {'indent': 2, 'name': '25 to 29 years'},
    # 'B01001036': {'indent': 2, 'name': '30 to 34 years'},
# Hispanic Population Size 
    # 'B03002012': {'indent': 1, 'name': 'Hispanic or Latino:'}
# Population size by Income Bracket
    # 'B19001011': {'indent': 1, 'name': '$50,000 to $59,999'},
    # 'B19001012': {'indent': 1, 'name': '$60,000 to $74,999'},

In [185]:
# initialize variables
income_list=[]
income_target_pop_size=0

age_list=[]
age_target_pop_size=0

hispanic_list=[]
hispanic_pop_target_size=0

metro_area_name_list=[]
metro_area_total_population=[]

#iterate through geoids and sum populations of interest.
for i in geoid:
    #record metro area name in a list
    metro_area_name_list.append(parsedjson['geography'][i]['name'])
    # aggregate age data by geoid
    age_target_pop_size=0
    for j in range(6):
        age_target_pop_size+=( #male tables
            parsedjson['data'][i]['B01001']['estimate']['B01001007']+
            parsedjson['data'][i]['B01001']['estimate']['B01001008']+
            parsedjson['data'][i]['B01001']['estimate']['B01001008']+
            parsedjson['data'][i]['B01001']['estimate']['B01001009']+
            parsedjson['data'][i]['B01001']['estimate']['B01001010']+
            parsedjson['data'][i]['B01001']['estimate']['B01001011']+
            parsedjson['data'][i]['B01001']['estimate']['B01001012']+
            # female tables 
            parsedjson['data'][i]['B01001']['estimate']['%s'%('B010010'+str(31+j))]
            )
    age_list.append(age_target_pop_size)

    # aggregate hispanic population data by geoid
    hispanic_pop_target_size=(
        parsedjson['data'][i]['B03002']['estimate']['B03002012']
        )
    hispanic_list.append(hispanic_pop_target_size)
    hispanic_pop_target_size=0

    # aggregate income bracket population data by geoid
    income_target_pop_size=(
        parsedjson['data'][i]['B19001']['estimate']['B19001011']+
        parsedjson['data'][i]['B19001']['estimate']['B19001012']
        )
    income_list.append(income_target_pop_size)
    income_target_pop_size=0

    #Add the metro area toal population to the list
    metro_area_total_population.append(parsedjson['data'][i]['B01001']['estimate']['B01001001'])

print(metro_area_name_list[0:5])
print(age_list[0:5])
print(hispanic_list[0:5])
print(income_list[0:5]) 
print(metro_area_total_population[0:5])

['Abilene, TX Metro Area', 'Akron, OH Metro Area', 'Albany, GA Metro Area', 'Albany, OR Metro Area', 'Albany-Schenectady-Troy, NY Metro Area']
[183792.0, 603500.0, 131300.0, 94602.0, 796467.0]
[38892.0, 13318.0, 4057.0, 10483.0, 42999.0]
[11570.0, 52446.0, 9529.0, 9273.0, 62094.0]
[169000.0, 703398.0, 153776.0, 121074.0, 881862.0]


In [186]:
df=pd.DataFrame()
df['metro_area']=metro_area_name_list
df['PARAM1_pop_in_target_income']=income_list
df['PARAM2_pop_in_target_age']=age_list
df['PARAM3_pop_hispanic']=hispanic_list
df['metro_area_total_pop']=metro_area_total_population
df.head(4)

,metro_area,PARAM1_pop_in_target_income,PARAM2_pop_in_target_age,PARAM3_pop_hispanic,metro_area_total_pop
0,"Abilene, TX Metro Area",11570.0,183792.0,38892.0,169000.0
1,"Akron, OH Metro Area",52446.0,603500.0,13318.0,703398.0
2,"Albany, GA Metro Area",9529.0,131300.0,4057.0,153776.0
3,"Albany, OR Metro Area",9273.0,94602.0,10483.0,121074.0


#Question 1: City with the Highest Population in our Target Income

In [187]:
df.sort_values(by=['PARAM1_pop_in_target_income'],ascending=False).head(1)

,metro_area,PARAM1_pop_in_target_income,PARAM2_pop_in_target_age,PARAM3_pop_hispanic,metro_area_total_pop
249,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",1050888.0,17561652.0,4863602.0,20192042.0


#Question 2: City with the Highest Population in our Target Age Range

In [188]:
df.sort_values(by=['PARAM2_pop_in_target_age'],ascending=False).head(1)

,metro_area,PARAM1_pop_in_target_income,PARAM2_pop_in_target_age,PARAM3_pop_hispanic,metro_area_total_pop
249,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",1050888.0,17561652.0,4863602.0,20192042.0


#Question 3: City with the Highest Hispanic Population

In [189]:
df.sort_values(by=['PARAM3_pop_hispanic'],ascending=False).head(1)

,metro_area,PARAM1_pop_in_target_income,PARAM2_pop_in_target_age,PARAM3_pop_hispanic,metro_area_total_pop
210,"Los Angeles-Long Beach-Anaheim, CA Metro Area",700681.0,12585318.0,5972751.0,13261538.0


# Calculate the proportion of the population reperesented by our target demographics

In [190]:
# we calculate the proportion of the population represented by our target market
# by didviding our target market population total by the metro area population
# we do this for each of our targeting parameters
df['income_percent']=df['PARAM1_pop_in_target_income']/df['metro_area_total_pop']
df['age_percent']=df['PARAM2_pop_in_target_age']/df['metro_area_total_pop']
df['hispanic_percent']=df['PARAM3_pop_hispanic']/df['metro_area_total_pop']
# We print out this data to view the results
df.head(3)

,metro_area,PARAM1_pop_in_target_income,PARAM2_pop_in_target_age,PARAM3_pop_hispanic,metro_area_total_pop,income_percent,age_percent,hispanic_percent
0,"Abilene, TX Metro Area",11570.0,183792.0,38892.0,169000.0,0.068462,1.087527,0.230130
1,"Akron, OH Metro Area",52446.0,603500.0,13318.0,703398.0,0.074561,0.857978,0.018934
2,"Albany, GA Metro Area",9529.0,131300.0,4057.0,153776.0,0.061967,0.853839,0.026383


In [191]:
# We notice that some of the percentages are incorrect as they are greater that 1 (more than 100%);
# Therefore, we subset the data so we only include metro areas where the proportions are acurate (where the proportions are <=1)
df=df[df['income_percent']<=1]
df=df[df['age_percent']<=1]
df=df[df['hispanic_percent']<=1]

# Finally, we can create a column with our mean percentage across all three categories 
# using a simple average of the three proportion columns we just created.
df['mean_target_percent']=(df['income_percent']+df['age_percent']+df['hispanic_percent'])/3

## View the edited data frame
df.head(3)

,metro_area,PARAM1_pop_in_target_income,PARAM2_pop_in_target_age,PARAM3_pop_hispanic,metro_area_total_pop,income_percent,age_percent,hispanic_percent,mean_target_percent
1,"Akron, OH Metro Area",52446.0,603500.0,13318.0,703398.0,0.074561,0.857978,0.018934,0.317158
2,"Albany, GA Metro Area",9529.0,131300.0,4057.0,153776.0,0.061967,0.853839,0.026383,0.314063
3,"Albany, OR Metro Area",9273.0,94602.0,10483.0,121074.0,0.076590,0.781357,0.086583,0.314843


#Question 4: City with the highest mean percentage across all three caregories


We can now sort our data frame by the mean_target_percent column to find the metro are with the highest average target population proportion. This may not be the best metric for as doing a simple average of percentages like this does treats all of our parameters the same; we may want to use a weighted average that scales the percentages before averaging based on the degree to which we prioritize each targeting parameter. Secondly, people could belong to multiple if not all of the of the demographic targeting filter we are using here; by using this method, we could create statistics that do not reflect the true in market population proportion.None the less, when we perform this calculation, we find that Jacksonville NC has the highest mean percentage across all three categories.

In [192]:
df.sort_values(by=['mean_target_percent'],ascending=False).head(1)

,metro_area,PARAM1_pop_in_target_income,PARAM2_pop_in_target_age,PARAM3_pop_hispanic,metro_area_total_pop,income_percent,age_percent,hispanic_percent,mean_target_percent
195,"Laredo, TX Metro Area",11460.0,251681.0,257482.0,269624.0,0.042504,0.933452,0.954967,0.643641
